<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 1. ติดตั้งไลบรารีที่ต้องใช้
!pip install gspread oauth2client notion-client python-dotenv

In [ ]:
# ✅ 2. Mount Google Drive และโหลด .env
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

env_path = '/content/drive/MyDrive/Automation_Keys/Notion_Expense/.env'
load_dotenv(dotenv_path=env_path)

# ✅ โหลดค่าจาก .env
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")
SHEET_ID = os.getenv("SHEET_ID")

In [66]:
# ✅ Dynamic Keyword สำหรับเดาค่าใช้จ่าย
dynamic_keywords = {
    "อาหาร": ["ข้าว", "แกง", "ก๋วยเตี๋ยว", "กาแฟ", "ชา", "ไข่", "นม", "ขนม", "โจ๊ก"],
    "ของใช้": ["สบู่", "แชมพู", "ผ้า", "ทิชชู่", "ยาสีฟัน", "ครีม"],
    "รถ": ["น้ำมัน", "รถเมล์", "จอดรถ", "เติมลม", "ค่าทางด่วน"],
    "สุขภาพ": ["ยา", "คลินิก", "หมอ", "ฟิตเนส"],
    "บริจาค": ["ทำบุญ", "บริจาค", "ถวาย", "ตักบาตร"],
    "เงินสด": ["ถอน", "เบิก", "รับเงิน", "โอน"],
}

# ✅ Mapping หมวด → Notion Database ID
category_to_db_id = {
    "อาหาร": os.getenv("NOTION_DB_ID_FOOD"),
    "ของใช้": os.getenv("NOTION_DB_ID_SHOPPING"),
    "รถ": os.getenv("NOTION_DB_ID_TRANSPORT"),
    "สุขภาพ": os.getenv("NOTION_DB_ID_HEALTH"),
    "บริจาค": os.getenv("NOTION_DB_ID_DONATION"),
    "เงินสด": os.getenv("NOTION_DB_ID_CASH"),
}

In [70]:
# ✅ BLOCK 3: เชื่อม Google Sheets ด้วย service_account.json ที่อยู่ใน Drive
# -------------------------------------------------------------------
# ใช้ไฟล์ JSON จาก Google Cloud เพื่อ authorize กับ Google Sheets API
# เราจะใช้ path เดียวกับที่ .env อยู่เพื่อความยืดหยุ่น
# -------------------------------------------------------------------

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os

# ✅ กำหนด scope สำหรับ Sheets และ Drive
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

# ✅ ใช้ path เดียวกับ .env → จะปลอดภัยและยืดหยุ่น
service_account_path = os.path.join(os.path.dirname(env_path), "service_account.json")

# ✅ ตรวจสอบว่าไฟล์นี้มีจริง
print("🗂️ service_account.json found ✅:", os.path.exists(service_account_path))

# ✅ โหลด credentials และเชื่อมต่อกับ Google Sheets API
creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_path, scope)
client = gspread.authorize(creds)

# ✅ เปิด Sheet ด้วย SHEET_ID จาก .env
sheet = client.open_by_key(SHEET_ID).sheet1

# ✅ ทดสอบชื่อ sheet
print("📄 Connected to Sheet:", sheet.title)

🗂️ service_account.json found ✅: True
📄 Connected to Sheet: Sheet1


In [ ]:
# ✅ 4. ฟังก์ชันแปลงข้อความ [หมวดหมู่] [จำนวนเงิน]
def parse_message(text):
    try:
        parts = text.strip().split()
        if len(parts) == 2:
            category = parts[0]
            amount = float(parts[1])
            return category, amount
        elif len(parts) == 1:
            return parts[0], 0.0  # เผื่อพิมพ์หมวดหมู่อย่างเดียว
        else:
            return None, None
    except:
        return None, None

In [ ]:
# ✅ 5. เชื่อมต่อ Notion API
from notion_client import Client

notion = Client(auth=NOTION_TOKEN)

def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": NOTION_DB_ID},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )

In [ ]:
# ✅ 6. ฟังก์ชัน Sync Google Sheet → Notion
def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        category, amount = parse_message(row['Message'])
        if not category or amount is None:
            print(f"❌ Format ผิดที่ row {idx+2}: {row['Message']}")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx+2, 4, "YES")
            print(f"✅ Synced row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")

In [ ]:
# ✅ 7. บันทึกข้อความจาก Line ลง Google Sheet
import datetime

def log_message(user_id, message):
    now = datetime.datetime.now().isoformat()
    sheet.append_row([now, user_id, message, "NO"])  # เพิ่มแถวใหม่ลง Google Sheet

In [71]:
# ✅ ฟังก์ชันส่งข้อมูลเข้า Notion แบบเดาหมวดหมู่จาก keyword
def push_to_notion_dynamic(message):
    # แปลงข้อความ เช่น "ข้าวมันไก่ 50" → ("ข้าวมันไก่", 50.0)
    category, amount = parse_message(message)

    if not category or amount is None:
        print(f"❌ รูปแบบข้อความไม่ถูกต้อง: '{message}'")
        return

    # เดาหมวดหมู่จาก keyword ในข้อความ
    guessed_category = find_category(category)

    if not guessed_category:
        print(f"⚠️ ไม่พบหมวดที่เหมาะกับ: '{category}' → ยังไม่สามารถส่งไป Notion ได้")
        return

    db_id = category_to_db_id.get(guessed_category)

    if not db_id:
        print(f"❌ ไม่พบ Database ID สำหรับหมวด '{guessed_category}'")
        return

    # สร้างข้อมูลใหม่ใน Notion
    notion.pages.create(
        parent={"database_id": db_id},
        properties={
            "Name": {"title": [{"text": {"content": message}}]},
            "Amount": {"number": amount}
        }
    )

    print(f"✅ ส่งข้อมูลสำเร็จ: '{message}' → หมวด '{guessed_category}'")

In [68]:
# ✅ Block 6 (เดิม) ยังมีพวก:
# def parse_message(...)
# def push_to_notion(...)
# def sync_sheet_to_notion(...)

# ⬇️ แล้วเพิ่มต่อท้ายได้เลยแบบนี้:
def find_category(text):
    for category, keywords in dynamic_keywords.items():
        if any(word in text for word in keywords):
            return category
    return None

In [81]:
# ✅ Sync Google Sheet → Notion โดยเดาหมวด และ log คำไม่รู้จัก
def sync_sheet_to_notion_dynamic():
    data = sheet.get_all_records()

    for idx, row in enumerate(data):
        # ✅ ข้ามแถวที่เคย sync ไปแล้ว
        if row['Synced?'].strip().upper() != "NO":
            continue

        message = row['Message']

        # ✅ แยกข้อความ เช่น "แกงเขียวหวาน 65" → ("แกงเขียวหวาน", 65.0)
        category, amount = parse_message(message)
        if not category or amount is None:
            print(f"❌ รูปแบบไม่ถูกต้อง ที่ row {idx+2}: '{message}'")
            continue

        # ✅ เดาหมวดจากข้อความ (หาใน dynamic_keywords)
        guessed_category = find_category(category)
        db_id = category_to_db_id.get(guessed_category)

        if not guessed_category or not db_id:
            print(f"⚠️ ไม่สามารถส่ง '{message}' ไปยัง Notion ได้ (หมวดไม่รู้จัก)")
            log_unknown_category(message, category, amount)
            continue

        try:
            notion.pages.create(
                parent={"database_id": db_id},
                properties={
                    "Name": {"title": [{"text": {"content": message}}]},
                    "Amount": {"number": amount}
                }
            )

            # ✅ อัปเดตสถานะใน Google Sheet เป็น "YES"
            sheet.update_cell(idx + 2, 4, "YES")
            print(f"✅ Synced row {idx+2}: '{message}' → หมวด '{guessed_category}'")

        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")


In [76]:
# ✅ Log คำที่ไม่รู้จักลง Sheet 'Unknown_Categories'
def log_unknown_category(message, parsed_category, amount):
    try:
        unknown_sheet = client.open_by_key(SHEET_ID).worksheet("Unknown_Categories")
    except:
        print("❌ ไม่พบ Sheet ชื่อ 'Unknown_Categories'")
        return

    now = datetime.datetime.now().isoformat()
    unknown_sheet.append_row([now, message, parsed_category or "", amount or ""])
    print(f"📥 Logged unknown: '{message}'")

In [74]:
# ✅ ฟังก์ชันสำหรับ log ข้อความที่ไม่เข้า keyword ไว้ใน Sheet 'Unknown_Categories'
def log_unknown_category(message, parsed_category, amount):
    try:
        unknown_sheet = client.open_by_key(SHEET_ID).worksheet("Unknown_Categories")
    except:
        print("❌ ไม่พบ Sheet ชื่อ 'Unknown_Categories'")
        return

    now = datetime.datetime.now().isoformat()
    unknown_sheet.append_row([now, message, parsed_category or "", amount or ""])
    print(f"📥 Logged unknown: '{message}'")

In [77]:
log_message("U001", "ลำโพงบลูทูธ 899")  # ไม่มีใน keyword
log_message("U002", "แชมพู 99")           # อยู่ในของใช้
sync_sheet_to_notion_dynamic()

✅ Logged: [2025-06-19T09:48:26.456626] U001 → ลำโพงบลูทูธ 899
✅ Logged: [2025-06-19T09:48:26.999672] U002 → แชมพู 99
⚠️ ไม่สามารถส่ง 'ข้าวผัดหมู 55' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'ข้าวผัดหมู 55'
⚠️ ไม่สามารถส่ง 'น้ำมัน 200' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'น้ำมัน 200'
⚠️ ไม่สามารถส่ง 'แชมพู 129' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'แชมพู 129'
⚠️ ไม่สามารถส่ง 'ลำโพงบลูทูธ 650' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'ลำโพงบลูทูธ 650'
⚠️ ไม่สามารถส่ง 'ลำโพงบลูทูธ 899' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'ลำโพงบลูทูธ 899'
⚠️ ไม่สามารถส่ง 'แชมพู 99' ไปยัง Notion ได้ (หมวดไม่รู้จัก)
📥 Logged unknown: 'แชมพู 99'


In [78]:
log_message("U003", "บะหมี่หมูแดง 120")  # ไม่มีใน keyword

✅ Logged: [2025-06-19T10:06:47.826558] U003 → บะหมี่หมูแดง 120


In [82]:
def log_and_sync_message(user_id, message):
    now = datetime.datetime.now().isoformat()
    sheet.append_row([now, user_id, message, "NO"])  # log ลง Google Sheet

    # แปลงข้อความ เช่น "บะหมี่หมูแดง 120"
    parsed_text, amount = parse_message(message)
    if not parsed_text or amount is None:
        return f"❌ ข้อความไม่ถูกต้อง: '{message}'"

    # เดาหมวดจาก keyword
    guessed_category = find_category(parsed_text)

    if not guessed_category:
        log_unknown_category(message, parsed_text, amount)
        return f"⚠️ ไม่สามารถบันทึก '{message}' ได้ (ไม่รู้จักหมวด)"

    db_id = category_to_db_id.get(guessed_category)
    if not db_id:
        return f"❌ ไม่พบ Database สำหรับหมวด '{guessed_category}'"

    # ส่งเข้า Notion
    try:
        notion.pages.create(
            parent={"database_id": db_id},
            properties={
                "Name": {"title": [{"text": {"content": message}}]},
                "Amount": {"number": amount}
            }
        )

        # ✅ อัปเดตว่า sync แล้วใน Google Sheet
        data = sheet.get_all_records()
        for i, row in enumerate(data):
            if row["Message"] == message and row["UserID"] == user_id:
                sheet.update_cell(i + 2, 4, "YES")  # คอลัมน์ D = "Synced?"
                break

        return f"✅ บันทึกเรียบร้อย: '{message}' → หมวด '{guessed_category}'"

    except Exception as e:
        return f"🚨 เกิดข้อผิดพลาดขณะส่งเข้า Notion: {e}"


In [83]:
response = log_and_sync_message("U003", "บะหมี่หมูแดง 120")
print(response)

📥 Logged unknown: 'บะหมี่หมูแดง 120'
⚠️ ไม่สามารถบันทึก 'บะหมี่หมูแดง 120' ได้ (ไม่รู้จักหมวด)


In [84]:
response = log_and_sync_message("U003", "ลำโพง 890")
print(response)

📥 Logged unknown: 'ลำโพง 890'
⚠️ ไม่สามารถบันทึก 'ลำโพง 890' ได้ (ไม่รู้จักหมวด)
